In [1]:
!pip install git+https://github.com/PennyLaneAI/pennylane

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import tensor

from pathlib import Path

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import embeddings as emb
from pennylane.templates import layers as lay

from typing import Union

import matplotlib.pyplot as plt

from music21 import converter, instrument, note, chord, stream
import glob, pickle, time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [46]:
from google.colab import files
uploaded = files.upload()
uploaded.keys()

Saving notes.pk to notes (1).pk


dict_keys(['notes.pk'])

In [15]:
# Midi.py

notes_dir = "notes.pk"

class Midi:
    def __init__(self, seq_length, device):
        self.seq_length = seq_length
        self.device = device

        if Path(notes_dir).is_file():
            self.notes = pickle.load(open(notes_dir, "rb"))
            # self.notes = pickle.loads(uploaded[notes_dir])
        else:
            self.notes = self.get_notes()
            pickle.dump(self.notes, open(notes_dir, "wb"))

        self.network_input, self.network_output = self.prepare_sequences(self.notes)
        print(f"Input shape: {self.network_input.shape}")
        print(f"Output shape: {self.network_output.shape}")

    def get_notes(self):
        """Get all the notes and chords from the midi files in the ./midi_songs directory"""
        # This is assuming that every interval between notes is the same (0.5)
        notes = []

        for file in glob.glob("midi_songs/*.mid"):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try:  # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except:  # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

        with open(notes_dir, "wb") as filepath:
            pickle.dump(notes, filepath)

        return notes

    def prepare_sequences(self, notes):
        """Prepare the sequences used by the Neural Network"""
        self.n_vocab = len(set(notes))

        # get all pitch names
        pitchnames = sorted(set(item for item in notes))

        # create a dictionary to map pitches to integers
        self.note_to_int = {note: number for number, note in enumerate(pitchnames)}
        self.int_to_note = {number: note for number, note in enumerate(pitchnames)}

        network_input = []
        network_output = []

        # create input sequences and the corresponding outputs
        for i in range(0, len(self.notes) - self.seq_length, 1):
            sequence_in = self.notes[i : i + self.seq_length]
            sequence_out = self.notes[i + self.seq_length]
            network_input.append([self.note_to_int[char] for char in sequence_in])
            network_output.append(self.note_to_int[sequence_out])

        n_patterns = len(network_input)

        # reshape the input into a format compatible with LSTM layers
        # So this is actuallyt (number of different inputs, sequence length, number of features)
        network_input = np.reshape(network_input, (n_patterns, self.seq_length, 1))
        # normalize input
        network_input = network_input / float(self.n_vocab)

        # network_output = to_categorical(network_output)

        return (tensor(network_input, device=self.device), tensor(network_output, device=self.device))

    def create_midi_from_model(self, prediction_output, filename):
        """ convert the output from the prediction to notes and create a midi file
        from the notes """
        offset = 0
        output_notes = []

        # create note and chord objects based on the values generated by the model
        for pattern in prediction_output:
            # pattern is a chord
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            # pattern is a note
            else:
                new_note = note.Note(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)

            # increase offset each iteration so that notes do not stack
            offset += 0.5

        midi_stream = stream.Stream(output_notes)

        midi_stream.write('midi', fp=filename)


In [4]:
# QLSTM.py

Embedding = Union[emb.AngleEmbedding, emb.AmplitudeEmbedding, emb.BasisEmbedding]
Layer = Union[
    lay.BasicEntanglerLayers,
    lay.ParticleConservingU1,
    lay.ParticleConservingU2,
    lay.RandomLayers,
    lay.StronglyEntanglingLayers,
]


class QLSTM(nn.Module):
    def quantum_op(
        self,
        wires,
        embedding: Embedding = emb.AngleEmbedding,
        layer: Layer = lay.BasicEntanglerLayers,
    ):
        def circuit_part(inputs, weights):
            embedding(inputs, wires=wires)
            layer(weights, wires=wires)
            return [qml.expval(qml.PauliZ(wires=w)) for w in wires]

        return circuit_part

    def __init__(
        self,
        input_size,
        hidden_size,
        n_qubits=4,
        n_qlayers=1,
        batch_first=True,
        return_sequences=False,
        return_state=False,
        backend="default.qubit",
        device="cpu"
    ):
        super(QLSTM, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.backend = backend  # "default.qubit", "qiskit.basicaer", "qiskit.ibm"
        self.device = device # "cpu", "cuda"

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state

        # self.dev = qml.device("default.qubit", wires=self.n_qubits)
        # self.dev = qml.device('qiskit.basicaer', wires=self.n_qubits)
        # self.dev = qml.device('qiskit.ibm', wires=self.n_qubits)
        # use 'qiskit.ibmq' instead to run on hardware

        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_input = [f"wire_input_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget)
        self.dev_input = qml.device(self.backend, wires=self.wires_input)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        self.qlayer_forget = qml.QNode(
            self.quantum_op(self.wires_forget), self.dev_forget, interface="torch"
        )

        self.qlayer_input = qml.QNode(
            self.quantum_op(self.wires_input), self.dev_input, interface="torch"
        )

        self.qlayer_update = qml.QNode(
            self.quantum_op(self.wires_update), self.dev_update, interface="torch"
        )

        self.qlayer_output = qml.QNode(
            self.quantum_op(self.wires_output), self.dev_output, interface="torch"
        )

        weight_shapes = {"weights": (n_qlayers, n_qubits)}
        print(f"weight_shapes = (n_qlayers, n_qubits) = ({n_qlayers}, {n_qubits})")

        self.clayer_in = torch.nn.Linear(self.concat_size, n_qubits)
        self.VQC = {
            "forget": qml.qnn.TorchLayer(self.qlayer_forget, weight_shapes).to(device),
            "input": qml.qnn.TorchLayer(self.qlayer_input, weight_shapes).to(device),
            "update": qml.qnn.TorchLayer(self.qlayer_update, weight_shapes).to(device),
            "output": qml.qnn.TorchLayer(self.qlayer_output, weight_shapes).to(device),
        }
        self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)
        # self.clayer_out = [torch.nn.Linear(n_qubits, self.hidden_size) for _ in range(4)]

    def forward(self, x, init_states=None):
        """
        x.shape is (batch_size, seq_length, feature_size)
        recurrent_activation -> sigmoid
        activation -> tanh
        """
        # Automatically assumes single batch
        x = x.to(self.device)
        if len(x.shape) == 2:
            x = x.reshape(1, x.shape[0], x.shape[1])
        
        if self.batch_first is True:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size, device=self.device) # hidden state (output)
            c_t = torch.zeros(batch_size, self.hidden_size, device=self.device) # cell state
        else:
            # for now we ignore the fact that in PyTorch you can stack multiple RNNs
            # so we take only the first elements of the init_states tuple init_states[0][0], init_states[1][0]
            h_t, c_t = init_states
            h_t = h_t[0]
            c_t = c_t[0]

        for t in range(seq_length):
            # get features from the t-th element in seq, for all entries in the batch
            x_t = x[:, t, :]

            # Concatenate input and hidden state
            v_t = torch.cat((h_t, x_t), dim=1).float().to(device)

            # match qubit dimension
            y_t = self.clayer_in(v_t).to(self.device)
            
            f_t = torch.sigmoid(
                self.clayer_out(self.VQC["forget"](y_t).to(self.device))
            ).to(self.device)  # forget block
            i_t = torch.sigmoid(self.clayer_out(self.VQC["input"](y_t).to(self.device)))  # input block
            g_t = torch.tanh(self.clayer_out(self.VQC["update"](y_t).to(self.device)))  # update block
            o_t = torch.sigmoid(
                self.clayer_out(self.VQC["output"](y_t).to(self.device)).to(self.device)
            ).to(self.device)  # output block

            c_t = (f_t * c_t) + (i_t * g_t)
            h_t = o_t * torch.tanh(c_t)

            hidden_seq.append(h_t.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()

        # Wow, such pseudo-keras!
        h_t, c_t = h_t.float(), c_t.float()

        if self.return_state:
            if self.return_sequences:
                return hidden_seq, (h_t, c_t)
            else:
                return (h_t, c_t)
        else:
            if self.return_sequences:
                return hidden_seq
            else:
                return h_t
    
    def predict(self, x, init_states=None):
        return self.forward(x, init_states)

In [457]:
# LSTMusic.py

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from QLTSM.qlstm import QLSTM

import numpy as np


class LSTMusic(nn.Module):
    def __init__(
        self,
        n_vocab=None,
        input_dim=1,
        hidden_dim=512,
        n_qubits=4,
        backend="default.qubit",
        device="cpu",
    ):
        super(LSTMusic, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        if n_qubits > 0:
            print(f"Generator will use Quantum LSTM running on backend {backend}")

            # self.model = nn.Sequential(
            #     QLSTM(
            #         input_size=input_dim,
            #         hidden_size=hidden_dim,
            #         n_qubits=n_qubits,
            #         backend=backend,
            #     ),
            #     nn.Dropout(0.3),
            #     QLSTM(
            #         input_size=input_dim,
            #         hidden_size=hidden_dim,
            #         n_qubits=n_qubits,
            #         backend=backend,
            #     ),
            #     nn.Dropout(0.3),
            #     QLSTM(
            #         input_size=input_dim,
            #         hidden_size=hidden_dim,
            #         n_qubits=n_qubits,
            #         backend=backend,
            #     ),
            #     nn.Linear(hidden_dim, hidden_dim),
            #     nn.Dropout(0.3),
            #     nn.Linear(hidden_dim, n_vocab),
            # )

            self.model = QLSTM(
                input_dim,
                hidden_dim,
                n_qubits=n_qubits,
                backend=backend,
                return_state=True,
                device=device,
            ).to(device)
        else:
            print("Generator will use Classical LSTM")
            self.model = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=3, dropout=0.3)

        # The linear layer that maps from hidden state space to tag space

    def forward(self, note_sequence):
        (h_t, (w_t, c_t)) = self.model(note_sequence)
        # print(type(h_t), type(c_t))
        # print(h_t.shape, w_t.shape, c_t[0].shape)
        scores = F.log_softmax(c_t[2], dim=1)
        # scores = F.log_softmax(c_t.reshape(c_t.shape[1], c_t.shape[2]).float(), dim=1)
        return scores

        # embeds = self.word_embeddings(sentence)
        # lstm_out, _ = self.model(embeds.view(len(sentence), 1, -1))
        # tag_logits = self.hidden2tag(lstm_out.view(len(sentence), -1))
        # tag_scores = F.log_softmax(tag_logits, dim=1)
        # return tag_scores

    def train(
        self,
        mode=True,
        inputs=None,
        outputs=None,
        n_epochs=None,
        cutoff: int = None,
        learning_rate=0.1,
    ):
        # Same as categorical cross entropy, who would've thought?!
        if mode == False:
            return
        loss_function = nn.NLLLoss()
        # print(self.parameters())
        optimizer = optim.SGD(self.model.parameters(), lr=learning_rate)

        if cutoff:
            inputs = inputs[:cutoff]
            outputs = outputs[:cutoff]

        history = {"loss": []}

        for epoch in range(n_epochs):
            counter = 0
            losses = []
            for note_series, next_note in zip(inputs, outputs):
                # Step 1. Remember that Pytorch accumulates gradients.
                # We need to clear them out before each instance
                self.zero_grad()

                # Step 2. Run our forward pass.
                # print(note_series)
                c_t = self(note_series.reshape(note_series.shape[0], 1, 1).float())

                # Step 4. Compute the loss, gradients, and update the parameters by
                #  calling optimizer.step()
                print(f"Result: {c_t}, Answer: {next_note}")
                loss = loss_function(c_t, next_note.reshape(1).long())
                loss.backward()
                optimizer.step()
                losses.append(float(loss))
                if counter % 1000 == 0:
                    print(f"On datapoint #{counter} out of {cutoff}")
                counter += 1

            avg_loss = np.mean(losses)
            history["loss"].append(avg_loss)
            print("Epoch {} / {}: Loss = {:.3f}".format(epoch + 1, n_epochs, avg_loss))
        return history

    def generate_notes(self, network_input, int_to_note, n_vocab, n_notes):
        """Generate notes from the neural network based on a sequence of notes"""
        # pick a random sequence from the input as a starting point for the prediction
        with torch.no_grad():
            start = np.random.randint(0, len(network_input) - 1)

            pattern = network_input[start]
            prediction_output = []

            # generate 500 notes
            for _ in range(n_notes):
                prediction_input = pattern.clone().detach().reshape(1, len(pattern), 1)
                # prediction_input = prediction_input / float(n_vocab)

                (h_t, prediction) = self.model.predict(prediction_input)

                index = prediction.argmax()
                result = int_to_note[int(index)]
                prediction_output.append(result)

                added_index = (index / n_vocab).reshape(1, 1)

                pattern = torch.cat((pattern, added_index), 0)
                # pattern.append(index)
                pattern = pattern[1 : len(pattern)]

            return prediction_output


In [459]:
seq_length = 25
n_epochs = 20
cutoff = 100
n_qubits = 0

model_name = f"lstm-seq{seq_length}-cut{cutoff}-epcs{n_epochs}-qu{n_qubits}"
model_str = f"{model_name}.pt"

In [498]:
print("Initialized Midi")
midi = Midi(seq_length, device)
# (midi.network_output[:200] == 137).sum(dim=0)
torch.mode(midi.network_output)

Initialized Midi
Input shape: torch.Size([44831, 25, 1])
Output shape: torch.Size([44831])


torch.return_types.mode(
values=tensor(233, device='cuda:0'),
indices=tensor(209, device='cuda:0'))

In [ ]:
print("Initialized LSTM")
lstm = LSTMusic(hidden_dim=midi.n_vocab, n_qubits=n_qubits, device=device).to(device)

if Path(model_str).is_file():
    print("Loading model")
    lstm.load_state_dict(torch.load(model_str))
    lstm.eval()
    # lstm = torch.load(model_str)
else:
    print("Training LSTM")
    train_history = lstm.train(
        True, midi.network_input, midi.network_output, n_epochs=n_epochs, cutoff=cutoff
    )
    torch.save(lstm.state_dict(), model_str)

In [ ]:
print(midi.network_output)

In [418]:
def generate_notes(self, network_input, int_to_note, n_vocab, n_notes):
        """Generate notes from the neural network based on a sequence of notes"""
        # pick a random sequence from the input as a starting point for the prediction
        with torch.no_grad():
            start = np.random.randint(0, len(network_input) - 1)
            print(f"start: {start}")

            pattern = network_input[start]
            prediction_output = []

            # generate 500 notes
            for _ in range(n_notes):
                prediction_input = pattern.clone().detach().reshape(1, len(pattern), 1)
                # prediction_input = prediction_input / float(n_vocab)

                prediction = self.forward(prediction_input.float())
                # print(h_t, prediction)

                index = prediction[-1].argmax()
                print(prediction[0].shape)
                print(torch.topk(prediction[0], k=10))
                result = int_to_note[int(index)]
                prediction_output.append(result)

                added_index = (index / n_vocab).reshape(1, 1)

                pattern = torch.cat((pattern, added_index), 0)
                # pattern.append(index)
                pattern = pattern[1 : len(pattern)]

            return prediction_output

In [140]:
# print(midi.int_to_note)

In [456]:
print("Generating notes")
notes = generate_notes(
    lstm, midi.network_input, midi.int_to_note, midi.n_vocab, n_notes=1
)
notes

Generating notes
start: 7612
torch.Size([308])
torch.return_types.topk(
values=tensor([-4.8998, -4.9637, -4.9940, -5.0890, -5.1090, -5.1886, -5.2079, -5.2201,
        -5.2274, -5.2382], device='cuda:0'),
indices=tensor([137, 282, 262, 294, 236, 233, 235, 140, 300, 306], device='cuda:0'))


['4.9']

In [24]:
print("Saving as MIDI file.")
midi.create_midi_from_model(notes, f"{model_name}_generated.mid")

Saving as MIDI file.
